In [1]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import plotly.graph_objects as go
import sys
sys.path.append('..')

#'Alessi_36', 'Collinder_135', 'NGC_2451B', 'OC_0450' Package imports

from oviz import Trace, TraceCollection, Animate3D

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

In [2]:
# Parameters for the ellipse
semi_major = 373 * u.pc  # Semi-major axis in parsecs
semi_minor = 233 * u.pc  # Semi-minor axis in parsecs
center_distance = 104 * u.pc  # Distance of the center from the Sun
center_longitude = 180.4 * u.deg  # Galactic longitude of the center
center_latitude = 0. * u.deg  # Galactic latitude (inclination interpreted as b)
inclination_angle = 17.2 * u.deg  # Galactic latitude (inclination interpreted as b)
ellipse_orientation = 296.1 * u.deg  # Orientation of the semi-major axis in Galactic longitude

# Number of points to model the ellipse
n_points = 100

# Generate the parametric ellipse in 2D (x', y')
theta = np.linspace(0, 2 * np.pi, n_points)
x_prime = semi_major * np.cos(theta)
y_prime = semi_minor * np.sin(theta)

# Rotate the ellipse by the orientation angle (ellipse_orientation)
rotation_angle = (ellipse_orientation - 45*u.deg).to(u.rad)  # Adjusted orientation
#rotation_angle = ellipse_orientation.to(u.rad)  # Adjusted orientation
x_rot = x_prime * np.cos(rotation_angle) - y_prime * np.sin(rotation_angle)
y_rot = x_prime * np.sin(rotation_angle) + y_prime * np.cos(rotation_angle)

rotation_angle_2 = inclination_angle.to(u.rad)
z_rot = y_rot * np.sin(rotation_angle_2)
y_rot = y_rot * np.cos(rotation_angle_2)

# rotation_angle_3 = 30*u.deg.to(u.rad)
# x_rot = x_rot * np.cos(rotation_angle_3) - z_rot * np.sin(rotation_angle_3)
# z_rot = x_rot * np.sin(rotation_angle_3) + z_rot * np.cos(rotation_angle_3)


# Convert the center to Galactic coordinates
galactic_center = SkyCoord(l=center_longitude, b=center_latitude, distance=center_distance, frame="galactic")
center_x = galactic_center.cartesian.x
center_y = galactic_center.cartesian.y
center_z = galactic_center.cartesian.z

# Translate the rotated ellipse to the center
x_gb = x_rot + center_x
y_gb = y_rot + center_y
z_gb = z_rot + center_z
#z_gb = np.full_like(x_gb, center_z)  # Constant Z value based on the center
x_gb = x_gb.value
y_gb = y_gb.value
z_gb = z_gb.value


In [3]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
#dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/mustache_work/figures/cubes/edenhofer_scatterized_4pc.csv')
dfv = dfv.loc[dfv['e'].between(0.003, 1)]
#dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 1
scatter_edenhofer = go.Scatter3d(
    x=dfv['x'].values[::ds_index],
    y=dfv['y'].values[::ds_index],
    z=dfv['z'].values[::ds_index],
    mode='markers',
    marker=dict(size=2,
                color='gray',
                symbol='circle',
                opacity=.02),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0

# gsh238 shell
dfv = pd.read_csv('/Users/cam/Downloads/gsh_238_region_edenhofer.csv')
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
scatter_gsh238 = go.Scatter3d(
    x=dfv['X'].values,
    y=dfv['Y'].values,
    z=dfv['Z'].values,
    mode='markers',
    marker=dict(size=1.5,
                color='gray',
                symbol='circle',
                opacity=.2),
    line = dict(color = 'gray', width = 0.),
    name='GSH238',
    visible = 'legendonly',
    hovertext='GSH238')
static_traces.append(scatter_gsh238)
static_traces_times.append([0.]) # only show at t=0

# read in leike map

hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

# annie IVS 
ivs = pd.read_csv('/Users/cam/Downloads/npix128boundary_mean_sigma10.csv')
ivs_out = pd.read_csv('/Users/cam/Downloads/npix128boundaryout_mean_sigma10.csv')
ivs = pd.concat([ivs, ivs_out])
ivs = ivs.sample(50000)
scatter_ivs = go.Scatter3d(
    x = ivs['x[pc]'].values,
    y = ivs['y[pc]'].values,
    z = ivs['z[pc]'].values,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='IVS (Annie)',
    visible = 'legendonly',
    hovertext='IVS')
# static_traces.append(scatter_ivs)
# scatter_ivs_times = np.arange(0, -5., -0.5)
# static_traces_times.append(scatter_ivs_times)

# Gould Belt
scatter_gb = go.Scatter3d(
    x=x_gb,
    y=y_gb,
    z=z_gb,
    mode='markers',
    marker=dict(size=2,
                color='white',
                symbol='circle',
                opacity=1.),
    name='Gould Belt',
    visible = 'legendonly',
    hovertext='Gould Belt')
# static_traces.append(scatter_gb)
# static_traces_times.append([0.]) # only show at t=0

In [4]:
#ages = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages_parallel.csv')
ages = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages_parallel_new.csv')
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
#df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 5) & 
    (df_hunt_full['U_err'] < 5) & 
    (df_hunt_full['V_err'] < 5) & 
    (df_hunt_full['W_err'] < 5) &
    (df_hunt_full['x'].between(-1000, 1000)) &
    (df_hunt_full['y'].between(-1000, 1000)) &
    (df_hunt_full['z'].between(-300, 300)) &
    (df_hunt_full['class_50'] >0.5) &
    (df_hunt_full['cst'] > 5)
    #(df_hunt_full['n_stars'] > 100)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(70, 200)]
df_hunt_older_2 = df_hunt_good.loc[df_hunt_good['age_myr'].between(150, 1000)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_70_myr.csv', index=False)
df_hunt_young.to_csv('/Users/cam/Downloads/hunt_catalog_0_15_myr.csv', index=False)

In [5]:
df_hunt_older = pd.merge(df_hunt_older, ages, on='name', how='left')
df_hunt_older['age_myr'] = df_hunt_older['age_chronos_mode'].fillna(df_hunt_older['age_myr'])

In [6]:
fam_list = [
    'HSC_1340',
    'CWNU_1111',
    'NGC_2232'
    'CWNU_1057',
    'HSC_1481',
    'ZHBJZ_1',
    'HSC_1373',
    'HSC_1460',
    'OCSN_50',
    'ASCC_24',
    'Theia_71'
]
a = df_hunt_good.loc[df_hunt_good['name'].isin(fam_list)]

In [7]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = pd.merge(df_hunt_gagne, ages, on='name', how='left')
df_hunt_gagne.loc[df_hunt_gagne['data_source'] == 'hunt', 'age_myr'] = df_hunt_gagne['age_chronos_mode'].fillna(df_hunt_gagne['age_myr'])
#df_hunt_gagne = df_hunt_gagne.loc[df_hunt_gagne['n_stars'] >= 100]

# # CHRONOS ages
# df_hunt_new_ages = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages_2.csv')
# df_hunt_gagne = df_hunt_gagne.merge(df_hunt_new_ages, on='name', how='left', suffixes=('', '_new_ages'))

# # use 'age_chronos_mode' for age if it exists, otherwise use 'age_myr'
# df_hunt_gagne['age'] = df_hunt_gagne['age_chronos_mode']
# df_hunt_gagne.loc[df_hunt_gagne['age'].isnull(), 'age'] = df_hunt_gagne['age_myr']
# df_hunt_gagne['age_myr'] = df_hunt_gagne['age']
# df_hunt_gagne.to_csv('/Users/cam/Downloads/cluster_sample_data_chornos_ages.csv', index=False)

df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [27.], 'n_stars' : [1]})


# Belt

In [8]:
star_forming_regions = {
    # 'Belt (Extra)': [
    #     'Roslund_5',
    #     'NGC_6793',
    #     'NGC_6633',
    #     'Ruprecht_147',
    #     'Messier_47'
    # ],
    'Orion': [
        'Briceno_1',
        'ASCC_21',
        'OCSN_59',
        'ASCC_19',
        'UBC_17a',
        'HSC_1633',
        'UPK_402',
    ],
    'Perseus': [
        'IC_348',
        'OC_0280',
        'HSC_1262',
        'Alessi-Teutsch_10'
    ],
    'Vela': [
        'Pozzo_1',
        'CWNU_1224',
        'CWNU_1083',
        'CWNU_1096',
        'OC_0479',
        'OC_0470',
    ],
    'Taurus': [
        'HSC_1318',
        'Theia_66',
        'Theia_54',
        'Theia_93'
    ],
    'Orion (comp A)': [
        'ASCC_20',
        'OCSN_61', 
        'ZHBJZ_1',
        'NGC_2232',
        'UPK_398',
        'OCSN_68',
        'CWNU_1072',
        'ASCC_18',
        'NGC_1980'
    ],
    'Orion (comp B)': [
        'UPK_422',
        'HSC_1653',
        'L_1641S',
        'HSC_1648'
    ],
    'UCL': [
        "UPK_640", 
        "UPK_606", 
        "OC_0666", 
        "HSC_2816", 
        "OCSN_92", 
        "HSC_2733", 
        "HSC_2636"
    ],
    'LCC': [
        "HSC_2523", 
        "HSC_2505", 
        "HSC_2468", 
        'Mamajek_1'
    ],
    'USCO': [
        "HSC_2907", 
        "Theia_67", 
        "HSC_2919",
        'OCSN_98',
        'OCSN_100',
        'OCSN_96',
        'HSC_2931',
        'HSC_2907'
    ],
    'Lacerta': [
        'Teutsch_39',
        'UBC_159',
        'OCSN_32',
        'Aveni-Hunter_1'
    ],
    'Cepheus': [
        'Theia_8',
        'Collinder_471',
        'Theia_5',
        'vdBergh_152',
        'ASCC_127'
    ],
    'Bridge': [
        'Stephenson_1',
        'ASCC_100',
        'Gaia_8',
        'HSC_474',
        'Teutsch_179',
        'HSC_415',
    ],
    'Cas-Tau': [
        'Melotte_20',
        'HSC_958',
        'ADS_16795',
        'HSC_1020',
        'Theia_232'
    ],
    'Trumpler 10': [
        'Trumpler_10'
    ],
    'Collinder 135': [
        'Collinder_135'
    ]

}

list_of_lists = list(star_forming_regions.values())


a_list = []
for name, cluster_list in star_forming_regions.items():
    a = df_hunt_full.loc[df_hunt_full['name'].isin(cluster_list)]
    a = a[['age_myr', 'U', 'V', 'W', 'x', 'y', 'z']]
    a = a.median()
    name = name
    a['name'] = name
    a_list.append(a)
df_sf_regions = pd.DataFrame(a_list)
sf_regions = Trace(df_sf_regions, data_name = 'Belt Associations (?)', min_size = 1.5, max_size = 15, color = 'cyan', opacity = 1., marker_style = 'circle', show_tracks = True)

In [9]:
# RW/Split/CS clusters
rw_region = pd.read_csv('/Users/cam/Downloads/rw_clusters.csv')
rw = rw_region.loc[rw_region.age_myr < 15]
remove = ['Theia_6', 'Teutsch_39', 'OCSN_32', 'Aveni-Hunter_1', 'IC_5146', 'Collinder_419', 'Dolidze_8', 'Theia_1920', 'Collinder_95', 'Mon_OB1-D', 'NGC_2264', 'UBC_159']
rw = rw[~rw['name'].isin(remove)]
rw = rw.rename(columns = column_renaming_dict)
#lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100']

# The Split
split_region = pd.read_csv('/Users/cam/Downloads/the_split_selection.csv')
split = split_region.loc[split_region.age_myr < 15]
split = split.rename(columns = column_renaming_dict)

# Cepheus spur
cepheus_region = pd.read_csv('/Users/cam/Downloads/cepheus_spur_region.csv')
cepheus_spur = cepheus_region.loc[cepheus_region.age_myr < 30]
cepheus_spur = cepheus_spur.rename(columns = column_renaming_dict)

# new spur?
spur_clusters = ['Pismis_5', 'Collinder_197', 'FSR_1421', 'BH_56', 'HSC_2144', 'ASCC_79', 'UPK_604', 'FSR_1694', 'NGC_6178', 'OC_0684', 'HSC_2911', 'CWNU_170', 'NGC_6383']
spur = df_hunt_full.loc[df_hunt_full['name'].isin(spur_clusters)]
spur = spur.rename(columns = column_renaming_dict)

In [10]:
# omuamua
x,y,z = 0., 0., 27.
U, V, W = -11.622, -22.461, -7.824
# U, V, W = 33.1, -6.8, 8.3
name = 'Oumuamua'
age = 60.

omuamua = pd.DataFrame({'name' : [name], 'family' : ['Oumuamua'], 'age_myr' : [age], 'U' : [U], 'V' : [V], 'W' : [W], 'x' : [x], 'y' : [y], 'z' : [z], 'n_stars' : [1]})

In [11]:
# new families
new_fam_1 = ['HSC_1373', 'OCSN_50', 'ASCC_24', 'HSC_1460', 'Theia_71', 'HSC_1340', 'CWNU_1057', 'CWNU_1111', 'CWNU_1092', 'HSC_1481', 'HSC_921', 'NGC_2232']
df_new_fam_1 = df_hunt_full.loc[df_hunt_full['name'].isin(new_fam_1)]

lacerta = ['HSC_661', 'vdBergh_152', 'HSC_705', 'OCSN_32', 'UBC_159', 'UPK_168', 'UPK_166', 'Teutch_39', 'Theia_100', 'HSC_699', 'CWNU_311', 'Teutsch_39', 'Aveni-Hunter_1', 'HSC_886']
df_lacerta = df_hunt_full.loc[df_hunt_full['name'].isin(lacerta)]

alphaPer_interloper = ['CWNU_1183', 'HSC_633', 'HSC_693', 'HSC_674', 'RSG_5', 'Theia_96', 'HSC_782', 'CWNU_519']
df_ap_interloper= df_hunt_full.loc[df_hunt_full['name'].isin(alphaPer_interloper)]

liu_clusters = [
    "ASCC_105", "OCSN_152", "OCSN_113", "OCSN_117", "Roslund_5", "UPK_82",
    "OCSN_16", "OCSN_18", "Stephenson_1", "UPK_101", "UPK_64", "UPK_72", 
    "UPK_78", "UPK_83", "OCSN_13", "OCSN_14", "UPK_70", "OCSN_22", 
    "OCSN_23", "RSG_5"
]
df_liu = df_hunt_full.loc[df_hunt_full['name'].isin(liu_clusters)]

In [12]:
#df_mustache_hi = pd.read_csv('/Users/cam/Downloads/mustache_with_3d_vels_connected_new.csv')
df_mustache_hi = pd.read_csv('/Users/cam/Downloads/mustache_with_3d_vels_connected_from_hi.csv')
df_mustache_hi = df_mustache_hi.sample(frac=.2)
df_mustache_hi = df_mustache_hi.rename(columns={'X' : 'x', 'Y' : 'y', 'Z' : 'z'})
df_mustache_hi['age_myr'] = 30.
df_mustache_hi['name'] = 'mustache_hi'

df_mustache_co = pd.read_csv('/Users/cam/Downloads/mustache_with_3d_vels_connected_from_co.csv')
df_mustache_co = df_mustache_co.rename(columns={'X' : 'x', 'Y' : 'y', 'Z' : 'z'})
df_mustache_co['age_myr'] = 30.
df_mustache_co['name'] = 'mustache_co'

mustache_hi = Trace(df_mustache_hi, data_name = 'Mustache (HI)', min_size = 10, max_size = 10, color = 'cyan', opacity = 0.4, marker_style = 'circle')
mustache_co = Trace(df_mustache_co, data_name = 'Mustache (CO)', min_size = 10, max_size = 10, color = 'orange', opacity = 0.4, marker_style = 'circle')

In [13]:
# Zari stars binned
zari = pd.read_csv('/Users/cam/Downloads/zari_stars_xyz_binned.csv')
zari['age_myr'] = 200
zari['name'] = 'Zari (OBA)'
zari_data = Trace(zari, data_name = 'Zari (OBA)', min_size = 5, max_size = 5, color = 'cyan', opacity = 0.4, marker_style = 'circle')

In [14]:
df_d_moving = pd.read_csv('/Users/cam/Downloads/edenhofer_scatterized_4pc_with_velocities.csv')
df_d_moving['age_myr'] = 30.
df_d_moving['name'] = 'Dust (Moving)'
d_moving = Trace(df_d_moving, data_name = 'Dust (Moving)', min_size = 0, max_size = 3, color = 'gray', opacity = 0.3, marker_style = 'circle')

In [15]:
# g1_names = pd.read_csv('/Users/cam/Downloads/g1.csv')
# g2_names = pd.read_csv('/Users/cam/Downloads/g2.csv')
# g3_names = pd.read_csv('/Users/cam/Downloads/g3.csv')
# g1 = df_hunt_full.loc[df_hunt_full['name'].isin(g1_names['name'])]
# g2 = df_hunt_full.loc[df_hunt_full['name'].isin(g2_names['name'])]
# g3 = df_hunt_full.loc[df_hunt_full['name'].isin(g3_names['name'])]


# g1_data = Trace(g1, data_name = 'Sirius MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
# g2_data = Trace(g2, data_name = 'Coma MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
# g3_data = Trace(g3, data_name = 'Pleiades MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)


In [16]:
df_hdbscan_resuls = pd.read_csv('/Users/cam/Downloads/clusters_hdbscanned_dec21.csv')
s_names = df_hdbscan_resuls.loc[df_hdbscan_resuls['final_label'] == 0]['name'].values
cb_names = df_hdbscan_resuls.loc[df_hdbscan_resuls['final_label'] == 2]['name'].values
ph_names = df_hdbscan_resuls.loc[df_hdbscan_resuls['final_label'] == 1]['name'].values

s = df_hunt_full.loc[df_hunt_full['name'].isin(s_names)]
cb = df_hunt_full.loc[df_hunt_full['name'].isin(cb_names)]
ph = df_hunt_full.loc[df_hunt_full['name'].isin(ph_names)]

s_data = Trace(s, data_name = 'Sirius MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
cb_data = Trace(cb, data_name = 'Coma MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)
ph_data = Trace(ph, data_name = 'Pleiades MG', min_size = 0, max_size = 7.5, color = '#6495ED', opacity = .8, marker_style = 'circle', show_tracks = False)

# Dark plot

In [17]:
# spurs
rw_data = Trace(rw, data_name = 'Radcliffe Wave', min_size = 3, max_size = 9, color = 'red', opacity = 1., marker_style = 'circle', show_tracks = False)
split_data = Trace(split, data_name = 'The Split', min_size = 0, max_size = 9, color = '#FFA07A', opacity = 1., marker_style = 'circle', show_tracks = False)
cepheus_spur_data = Trace(cepheus_spur, data_name = 'Cepheus Spur', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle', show_tracks = False)
new_spur = Trace(spur, data_name = 'Sag-Vela Spur (new?)', min_size = 0, max_size = 9, color = '#90EE90', opacity = 1., marker_style = 'circle', show_tracks = False)

# families
sample_data = Trace(df_hunt_gagne, data_name = 'Clean Clusters Sample', min_size = 0, max_size = 9, color = 'gray', opacity = 1., marker_style = 'circle')
ap_data = Trace(ap, data_name = '⍺Per', min_size = 0, max_size = 9, color = 'violet', opacity = 1, marker_style = 'circle', show_tracks = False)
m6_data = Trace(m6, data_name = 'M6', min_size = 0, max_size = 9, color = 'cyan', opacity = 1, marker_style = 'circle', show_tracks = False)
cr135_data = Trace(cr135, data_name = 'Cr135', min_size = 0, max_size = 9, color = 'orange', opacity = 1, marker_style = 'circle', show_tracks = False)
gamma_vel_data = Trace(gamma_vel, data_name = 'Gamma Vel', min_size = 0, max_size = 9, color = 'red', opacity = 1., marker_style = 'circle')

# new families
unnamed_family_1= Trace(df_new_fam_1, data_name = 'Unnamed family 1', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')
lacerta = Trace(df_lacerta, data_name = 'Lacerta', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')
ap_interloper = Trace(df_ap_interloper, data_name = '⍺Per interloper', min_size = 0, max_size = 7, color = 'gray', opacity = 1., marker_style = 'circle')

young_data = Trace(df_hunt_young, data_name = 'Clusters (< 15 Myr)', min_size = 0., max_size = 9, color = 'white', opacity = 0.5, marker_style = 'circle')
older_data = Trace(df_hunt_older, data_name = 'Clusters (Age = (0, 200 Myr))', min_size = 0., max_size = 5, color = 'gray', opacity = 1., marker_style = 'circle')
older_data_2 = Trace(df_hunt_older_2, data_name = 'Clusters (Age = (150, 500 Myr))', min_size = 0., max_size = 3, color = 'gray', opacity = 1., marker_style = 'circle')
sun_data = Trace(sun, data_name = 'Sun', min_size = 0, max_size = 8, color = 'yellow', opacity = 1., marker_style = 'circle')

# liu
liu_data = Trace(df_liu, data_name = 'Liu', min_size = 0, max_size = 12, color = 'white', opacity = 1., marker_style = 'circle')

#omuamua
omuamua_data = Trace(omuamua, data_name = 'Oumuamua', min_size = 0, max_size = 20., color = 'pink', opacity = 1., marker_style = 'circle')

# mate data
df_mate = pd.read_csv('/Users/cam/Downloads/lacerta_members_3dvelocities (1).csv')
df_mate = df_mate.rename(columns={'label' : 'name', 'x_helioc' : 'x', 'y_helioc' : 'y', 'z_helioc' : 'z', 'u_hel' : 'U', 'v_hel' : 'V', 'w_hel' : 'W'})
df_mate = df_mate.groupby('name').median().reset_index()
lacerta = Trace(df_mate, data_name = 'Lacerta', min_size = 0, max_size = 9, color = 'cyan', opacity = 1., marker_style = 'circle')

#df_pleiades = df_hunt_good.loc[df_hunt_good['name'] == 'Melotte_22']
df_interesting_clusters = df_hunt_full.loc[df_hunt_full['name'].isin(['Melotte_22', 'Melotte_111', 'IC_4725', 'Melotte_25', 'NGC_2632', 'NGC_2516', 'Collinder_394', 'NGC_2287'])]
interesting_clusters = Trace(df_interesting_clusters, data_name = 'Interesting Clusters', min_size = 0, max_size = 15, color = '#FF7F50', opacity = 1., marker_style = 'circle')

sc = TraceCollection([sf_regions, sample_data, young_data, older_data, older_data_2,sun_data, ap_data, m6_data, cr135_data, gamma_vel_data, rw_data, split_data, cepheus_spur_data, new_spur, unnamed_family_1, lacerta, ap_interloper, omuamua_data, interesting_clusters, s_data, ph_data, cb_data])
grouping_dict = {
    #'The Mustache and LB clusters' : ['Mustache (HI)', 'Mustache (CO)', '⍺Per', 'Radcliffe Wave', 'Sun'],
    'Clusters' : ['Clusters (Age = (0, 200 Myr))', 'Clusters (Age = (150, 500 Myr))', 'Sirius MG', 'Coma MG', 'Pleiades MG', 'Interesting Clusters', 'Clean Clusters Sample', '⍺Per', 'M6', 'Cr135', 'Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sun'],
    'Cluster families' : ['Belt Associations (?)', 'Clusters (< 15 Myr)', 'Clean Clusters Sample', '⍺Per', 'M6', 'Cr135', '𝝲Vel', 'Sun'],
    'Dust structures' : ['Clusters (< 15 Myr)', 'Radcliffe Wave', 'The Split', 'Cepheus Spur', 'Sag-Vela Spur (new?)', 'Sun']
    #'Omuamua with Alpha Per' : ['Oumuamua', '⍺Per', 'Sun']
}
#grouping_dict = None

In [18]:
df_interesting_clusters.name

1347    Collinder_394
4300          IC_4725
4423       Melotte_22
4424       Melotte_25
4430      Melotte_111
4537         NGC_2287
4585         NGC_2516
4598         NGC_2632
Name: name, dtype: object

In [19]:
#xyz_widths = (1250, 1250, 400)
xyz_widths = (5000, 5000, 400)

plot_3d = Animate3D(
    data_collection = sc, 
    #xyz_widths = xyz_widths, 
    xyz_ranges = ([-10000, 10000], [-10000, 10000], [-500, 500]),
    figure_theme = 'dark', 
    trace_grouping_dict=grouping_dict)

save_name = '/Users/cam/Downloads/gal_movie_test.html'
time_int = np.round(np.arange(-100, 100, 1), 1)

fig3d = plot_3d.make_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=True,
    #focus_group = None,
    focus_group = 'Pleiades MG',
    #focus_group = 'Gamma Vel',
    #focus_group = '⍺Per',
    #focus_group = 'Cr135',
    fade_in_time = 10, # Myr,
    fade_in_and_out = False,
    show_gc_line = True
)